So ideally this code will work similarly to the other diagnostic-evaluation codes but now only in reference to SSW-specific scenarios. 

In [1]:
#import packages
import datetime as dt
import numpy as np
import xarray as xr
import scipy.stats
import pickle
import pandas as pd

In [2]:
##N20 related gph and 850 temp files ... will just use this for getting cap values. 
infile = open("../../../New_ERAfiles/N20_gph_NEW.p", 'rb')
gph = pickle.load(infile)
infile.close()

infile = open("../../../New_ERAfiles/N20_temp850_NEW.p", 'rb')
temp = pickle.load(infile)
infile.close()

infile = open("../../../New_ERAfiles/N20_temp850_time_NEW.p", 'rb')
temp_time = pickle.load(infile)
infile.close()

infile = open("../../../New_ERAfiles/N20_gph_time_NEW.p", 'rb')
gph_time = pickle.load(infile)
infile.close()

infile = open("../../../New_ERAfiles/N20_gph_lat_NEW.p", 'rb')
gph_lat = pickle.load(infile)
infile.close()

infile = open("../../../New_ERAfiles/N20_gph_lon_NEW.p", 'rb')
gph_lon = pickle.load(infile)
infile.close()

In [3]:
##N20 cluster labels
infile = open("../ClusterLabels/NEW_GPHcluster_labels_HOURLY.p", 'rb') 
N20label_H = pickle.load(infile)
infile.close()

infile = open("../ClusterLabels/N20_HOURLY_ANOMcluster_labels.p", 'rb') 
NAlabel_H = pickle.load(infile)
infile.close()

In [4]:
##All lat cluster labels
infile = open("../ClusterLabels/ALL_HOURLY_GPHcluster_labels.p", 'rb') 
ALLlabel_H = pickle.load(infile)
infile.close()

infile = open("../ClusterLabels/ALL_HOURLY_ANOMcluster_labels.p", 'rb') 
AAlabel_H = pickle.load(infile)
infile.close()

In [5]:
##import relevant Elliptical Value Files 
#old root is ../../Ellipse_ERAI_data/
infile = open("../../../New_EllipseVals/ephi10_NEW.p", 'rb')
ephi10 = pickle.load(infile)
infile.close()

infile = open("../../../New_EllipseVals/ephi_ratio10_NEW.p", 'rb')
rat10 = pickle.load(infile)
infile.close()

infile = open("../../../New_EllipseVals/ephi_size10_NEW.p", 'rb')
size10 = pickle.load(infile)
infile.close()

infile = open("../../../New_EllipseVals/ephi_cenlat10_NEW.p", 'rb')
cenlat10 = pickle.load(infile)
infile.close()

infile = open("../../../New_EllipseVals/ephi_cenlon10_NEW.p", 'rb')
cenlon10 = pickle.load(infile)
infile.close()

infile = open("../../../New_EllipseVals/ephi_wind10_NEW.p", 'rb')
wind10 = pickle.load(infile)
infile.close()

In [6]:
infile = open("../N20_UW_lat.p", 'rb')
unweighted_lat = pickle.load(infile)
infile.close()

infile = open("../N20_UW_lon.p", 'rb') 
unweighted_lon = pickle.load(infile)
infile.close()

In [7]:
##need to do something to code in dates for x-axis 
year = [2000,2001,2002,2003,2005,2006,2007,2008,2009,2012,2017,2018]
sswcl = [1,2,3,4,6,7,8,9,10,13,18,19]

normal_year = pd.date_range('2000-11-01','2001-04-01', 
              freq='6H').strftime("%m/%d").tolist()
leap_year = pd.date_range('1999-11-01','2000-04-01', 
              freq='6H').strftime("%m/%d").tolist()

dates = [(385,528),(176,319),(256,399),(208,351),(268,411),(404,547),(396,539),(280,423),(344,487),(212,355),(356,499),(192,335)]
print(dates[0])

ephi10[0,int(dates[0][0]):int(dates[0][1])]
xt = leap_year[int(dates[0][0]):int(dates[0][1])]

(385, 528)


In [8]:
len(xt)

143

In [9]:
##separate out metric values

In [10]:
#non-stacked
##creating DataArray for gph
gph = xr.DataArray(data= gph, 
                       dims = ["time","lat","lon"],
                       coords = dict(
                           time = gph_time,
                           lat = unweighted_lat,
                           lon = unweighted_lon)
                      )

In [25]:
gp = gph.sel(lat=slice(60,90)).values
gp1 = np.nanmean(gp,axis = 1)
gp2 = np.nanmean(gp1,axis = 1)
gp2.shape

gp_f = np.reshape(gp2,(20,608))

In [26]:
#calculate daily mean values
daily_mean_gph = gph.groupby('time.dayofyear').mean()
#test gph anomaly
daily_anom_gph = gph.groupby('time.dayofyear') - daily_mean_gph

In [27]:
ga = daily_anom_gph.sel(lat=slice(60,90)).values
ga1 = np.nanmean(ga,axis = 1)
ga2 = np.nanmean(ga1,axis = 1)
ga2.shape

ga_f = np.reshape(ga2,(20,608))

In [28]:
#non-stacked
##creating DataArray for gph
temp = xr.DataArray(data= temp, 
                       dims = ["time","lat","lon"],
                       coords = dict(
                           time = temp_time,
                           lat = unweighted_lat,
                           lon = unweighted_lon)
                      )

In [29]:
tm = temp.sel(lat=slice(60,90)).values

In [30]:
tm1 = np.nanmean(tm,axis = 1)
tm2 = np.nanmean(tm1,axis = 1)
tm2.shape

(12160,)

In [31]:
tm_f = np.reshape(tm2,(20,608))

In [32]:
#calculate daily mean values
daily_mean_temp = temp.groupby('time.dayofyear').mean()
#test gph anomaly
daily_anom_temp = temp.groupby('time.dayofyear') - daily_mean_temp

In [33]:
ta = daily_anom_temp.sel(lat=slice(60,90)).values

In [34]:
ta1 = np.nanmean(ta,axis = 1)
ta2 = np.nanmean(ta1,axis = 1)
ta2.shape

(12160,)

In [35]:
ta_f = np.reshape(ta2,(20,608))

In [37]:
##SSW Averaging Timelines
ssw_wind = np.empty((12,143))
ssw_wind[:] = np.nan

ssw_rat = np.empty((12,143))
ssw_rat[:] = np.nan

ssw_size = np.empty((12,143))
ssw_size[:] = np.nan

ssw_cenl = np.empty((12,143))
ssw_cenl[:] = np.nan

ssw_ep= np.empty((12,143))
ssw_ep[:] = np.nan

ssw_t= np.empty((12,143))
ssw_t[:] = np.nan

for i in range(len(year)):
    d1 = int(dates[i][0])
    d2 = int(dates[i][1])
    
    ##diagnostics
    
    w = wind10[sswcl[i],d1:d2] #wind
    ssw_wind[i] = w[:]
    
    rat = rat10[sswcl[i],d1:d2]
    ssw_rat[i] = rat[:]
    
    cl = cenlat10[sswcl[i],d1:d2]
    ssw_cenl[i] = cl[:]
    
    sz = size10[sswcl[i],d1:d2]
    ssw_size[i] = sz[:]
    
    ep = ephi10[sswcl[i],d1:d2]
    ssw_ep[i] = ep[:]
    
    tm = tm_f[sswcl[i],d1:d2]
    ssw_t[i] = tm[:]